<a href="https://colab.research.google.com/github/Anu1996rag/Neural_Networks/blob/master/OptimizingCNN_using_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This file requires tensorflow of version 2.1.0 

In [ ]:
!pip install keras-tuner
!pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 25kB/s 
     |████████████████████████████████| 450kB 48.7MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 3.9MB 46.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=4aaf50af997912d1cc4cf2fc281ff9891b6501b958f32629763e580ca05cbed2
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninsta

In [ ]:
# Importing relevant libraries 

import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.1.0


In [ ]:
#loading the dataset 

fashion_mnist=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# converting the image pixel values from 0 to 1 and reshaping the image shapes

train_images=train_images/255.0
test_images=test_images/255.0

train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
# builiding a generic function of model building 

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
# using KerasTuner for hyperparameter optimization

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
# this will run for 5 times for 3 epochs each 
''' The Trial Summary shows the optimal parameter values for each trial '''

tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 19s 345us/sample - loss: 0.4760 - accuracy: 0.8337 - val_loss: 0.3771 - val_accuracy: 0.8595
Epoch 2/3
54000/54000 [==============================] - 12s 216us/sample - loss: 0.3394 - accuracy: 0.8751 - val_loss: 0.3197 - val_accuracy: 0.8792
Epoch 3/3
54000/54000 [==============================] - 12s 217us/sample - loss: 0.3088 - accuracy: 0.8854 - val_loss: 0.3890 - val_accuracy: 0.8673


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 14s 251us/sample - loss: 0.5375 - accuracy: 0.8174 - val_loss: 0.4289 - val_accuracy: 0.8422
Epoch 2/3
54000/54000 [==============================] - 13s 245us/sample - loss: 0.3915 - accuracy: 0.8558 - val_loss: 0.4149 - val_accuracy: 0.8523
Epoch 3/3
54000/54000 [==============================] - 13s 245us/sample - loss: 0.3654 - accuracy: 0.8644 - val_loss: 0.3688 - val_accuracy: 0.8623


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 13s 233us/sample - loss: 0.4870 - accuracy: 0.8338 - val_loss: 0.3722 - val_accuracy: 0.8630
Epoch 2/3
54000/54000 [==============================] - 12s 226us/sample - loss: 0.3473 - accuracy: 0.8734 - val_loss: 0.3339 - val_accuracy: 0.8802
Epoch 3/3
54000/54000 [==============================] - 12s 226us/sample - loss: 0.3157 - accuracy: 0.8834 - val_loss: 0.3427 - val_accuracy: 0.8755


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 13s 234us/sample - loss: 0.5090 - accuracy: 0.8203 - val_loss: 0.3858 - val_accuracy: 0.8520
Epoch 2/3
54000/54000 [==============================] - 12s 226us/sample - loss: 0.3774 - accuracy: 0.8598 - val_loss: 0.3753 - val_accuracy: 0.8593
Epoch 3/3
54000/54000 [==============================] - 12s 229us/sample - loss: 0.3388 - accuracy: 0.8734 - val_loss: 0.4056 - val_accuracy: 0.8608


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 13s 233us/sample - loss: 0.4634 - accuracy: 0.8374 - val_loss: 0.3365 - val_accuracy: 0.8732
Epoch 2/3
54000/54000 [==============================] - 12s 225us/sample - loss: 0.3406 - accuracy: 0.8726 - val_loss: 0.3429 - val_accuracy: 0.8777
Epoch 3/3
54000/54000 [==============================] - 12s 223us/sample - loss: 0.3089 - accuracy: 0.8846 - val_loss: 0.3470 - val_accuracy: 0.8775


INFO:tensorflow:Oracle triggered exit


In [ ]:
# finding out the best model with the best optimal hyperparameters from all the trials 

model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,032,618
Trainable params: 3,032,618
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# using the above best model to fit the data and getting the accuracy values 

model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 13s 244us/sample - loss: 0.3209 - accuracy: 0.8832 - val_loss: 0.4365 - val_accuracy: 0.8500
Epoch 5/10
54000/54000 [==============================] - 13s 235us/sample - loss: 0.2849 - accuracy: 0.8934 - val_loss: 0.3699 - val_accuracy: 0.8673
Epoch 6/10
54000/54000 [==============================] - 13s 235us/sample - loss: 0.2715 - accuracy: 0.8997 - val_loss: 0.3948 - val_accuracy: 0.8768
Epoch 7/10
54000/54000 [==============================] - 13s 236us/sample - loss: 0.2524 - accuracy: 0.9064 - val_loss: 0.4079 - val_accuracy: 0.8768
Epoch 8/10
54000/54000 [==============================] - 13s 235us/sample - loss: 0.2338 - accuracy: 0.9147 - val_loss: 0.4706 - val_accuracy: 0.8708
Epoch 9/10
54000/54000 [==============================] - 13s 236us/sample - loss: 0.2276 - accuracy: 0.9184 - val_loss: 0.4819 - val_accuracy: 0.8732
Epoch 10/10
54000/54000 [====================